In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options as ChromeOptions
from time import sleep
import pandas as pd

In [ ]:
options = ChromeOptions()
options.set_capability('pageLoadStrategy', 'none')
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(options=options)

In [ ]:
driver.get('https://www.taobao.com/')
wait = WebDriverWait(driver, 5, 0.5)
search_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#q')))
search_input.send_keys('数字电路 书')
search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type="submit"]')))
search_button.click()

In [ ]:
total_page = int(wait.until(EC.presence_of_element_located(
    (By.XPATH, '//div[@class="next-pagination-pages"]//span[@class="next-pagination-display"]'))).text.split('/')[-1])
map_price = lambda price: price.find_element(
    By.CLASS_NAME, 'Price--priceInt--ZlsSi_M').text + price.find_element(By.CLASS_NAME, 'Price--priceFloat--h2RR0RK').text
titles = []
prices = []
shops = []
for page_num in range(1, total_page + 1):
    page_input = wait.until(EC.presence_of_element_located(
        (By.XPATH, '//div[@class="next-pagination-pages"]//span[@class="next-input next-medium next-pagination-jump-input"]/input')))
    page_input.clear()
    page_input.send_keys(page_num)
    page_button = wait.until(EC.element_to_be_clickable(
        (By.XPATH, '//div[@class="next-pagination-pages"]//button[@class="next-btn next-medium next-btn-normal next-pagination-jump-go"]')))
    page_button.click()
    sleep(5)
    element_list = driver.find_element(
        By.CLASS_NAME, 'Content--contentInner--QVTcU0M')
    title_list = element_list.find_elements(
        By.XPATH, '//div[@class="Title--title--jCOPvpf "]/span')
    price_list = element_list.find_elements(
        By.XPATH, '//div[@class="Price--priceWrapper--Q0Dn7pN "]/div[@style="margin-right: 8px;"]')
    shop_list = element_list.find_elements(
        By.XPATH, '//div[@class="ShopInfo--TextAndPic--yH0AZfx"]/a[@class="ShopInfo--shopName--rg6mGmy"]')
    titles += [title.text for title in title_list]
    prices += [map_price(price) for price in price_list]
    shops += [shop.text for shop in shop_list]

In [ ]:
pd.DataFrame({'标题': titles, '价格（元）': prices, '店铺': shops}).to_excel('taobao.xlsx', index=False)

In [ ]:
pd.DataFrame({'标题': titles, '价格（元）': prices, '店铺': shops}).to_csv('taobao.csv', index=False)